In [34]:
import numpy as np
import plotly.graph_objects as go
from gpcam import GPOptimizer

# Plotting the results on a 3D plot
def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y, z=z))
    if data is not None:
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers',marker=dict(size=12,
                                                              color=data[:,2],         # Set color equal to a variable
                                                              #colorscale='Viridis',   # Choose a colorscale
                                                              opacity=0.8)
                                   )
        )

    fig.update_layout(title='Plot', autosize=True,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90),scene=dict(xaxis_title='Temperature', yaxis_title='Cycle', zaxis_title='Energy'))


    fig.show()

## Booth function
$$ 
f(x_1, x_2) = (x_1 + 2x_2 - 7)^2 + (2x_1 + x_2 -5)^2
$$

In [46]:
array1 = np.column_stack((np.full((100,), -5), np.arange(-10, 10, 0.2)))
array2 = np.column_stack((np.full((100,), 0), np.arange(-10, 10, 0.2)))
array3 = np.column_stack((np.full((100,), 5), np.arange(-10, 10, 0.2)))

x_data = np.vstack((array1, array2, array3))

x1 = x_data[:,0]
x2 = x_data[:,1]

y_data = (x1 + 2*x2 - 7)**2 + (2*x1 + x2 - 5)**2

error = np.random.normal(loc=200, scale=50, size=y_data.shape)
y_data = y_data + error

In [37]:
# Define the grid size
n = 100

# Design Space Limits
c_low = -10
c_high = 10
temp_low = -10
temp_high = 10

# Create a design space
y_space = np.linspace(c_low, c_high, n)
x_space = np.linspace(temp_low, temp_high, n)
x_space, y_space = np.meshgrid(x_space, y_space)

# Reshape the arrays into a 2-column array with 10000 rows
my_space = np.vstack((x_space.reshape(-1), y_space.reshape(-1))).T

In [38]:
print(my_space.shape)

(10000, 2)


In [39]:
def get_distance_matrix(x1,x2):
    d = np.zeros((len(x1),len(x2)))
    for i in range(x1.shape[1]):
        d += (x1[:,i].reshape(-1, 1) - x2[:,i])**2
    return np.sqrt(d + 1e-16)

def s(x, hps):
    slope = hps[3]
    offset = hps[4]
    o = slope * x[:,0] + offset
    return np.diag(o)

def mean(x, hps):
    y = -(x[:,1]/hps[5])**2 - hps[6]*x[:,1]*(hps[7] - x[:,0]) 
    return y

In [48]:
# hps[0] 1, 2 for kernel. 3,4 for noise, 5,6,7 for meanf

bounds = np.array([[0.001,100]    # in kernel, signal variance
                   ,[0.001,100]   # in kernel, length scale
                   ,[0.001,100]   # in kernel, length scale 2
                   ,[0.1,10]      # s, slope
                   ,[0.1,10]      # s, offset
                   ,[0, 2500]     # mean, 
                   ,[0, 2500]     # mean
                   
                   ])

my_trained_hps = np.array([0.01      # in kernel
                           , 0.01    # in kernel
                           , 0.01    # in kernel
                           , 1       # s
                           , 1       # s
                           , 80      # mean
                           , 4000    # mean
                           ])

my_gpo = GPOptimizer(x_data, y_data,
                      init_hyperparameters=my_trained_hps,
                      # hyperparameter_bounds=bounds,
                      # gp_kernel_function= my_kernel,
                      # gp_mean_function=mean,
                      # gp_noise_function=s
                      )


my_gpo.train(hyperparameter_bounds=bounds)

#print("hps: ", my_gpo.hyperparameters)
f = my_gpo.posterior_mean(my_space)["f(x)"]
v = my_gpo.posterior_covariance(my_space, add_noise=True)["v(x)"]


f_re = f.reshape(100,100)

c:\Users\MichelFaloughi\AppData\Local\Programs\Python\Python311\Lib\site-packages\fvgp\gp.py:279: UserWarning:

No noise function or measurement noise provided. Noise variances will be set to 1% of mean(y_data).



In [47]:
plot(x_space, y_space, f_re, data = np.column_stack([my_gpo.x_data, my_gpo.y_data]))